In [ ]:
import zipfile
import os

# Set the path to the uploaded ZIP file
zip_path = '/content/Clutch.v3i.yolov11.zip'  # Update this if your file has a different name
extract_path = '/content/dataset'  # Destination folder

# Create the destination directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset extracted to {extract_path}")

Dataset extracted to /content/dataset


In [ ]:
import albumentations as A
import os
import cv2
from glob import glob
from albumentations.pytorch import ToTensorV2

# Example YOLO -> Albumentations conversion helper
def yolo_to_albumentations(box, img_w, img_h):
    x_center, y_center, width, height = box
    x_min = (x_center - width / 2) * img_w
    y_min = (y_center - height / 2) * img_h
    x_max = (x_center + width / 2) * img_w
    y_max = (y_center + height / 2) * img_h
    return [x_min, y_min, x_max, y_max]

def albumentations_to_yolo(box, img_w, img_h):
    x_min, y_min, x_max, y_max = box
    x_center = ((x_min + x_max) / 2) / img_w
    y_center = ((y_min + y_max) / 2) / img_h
    width = (x_max - x_min) / img_w
    height = (y_max - y_min) / img_h
    return [x_center, y_center, width, height]

# Albumentations transform with bbox support
transform = A.Compose([
    A.Rotate(limit=15, p=0.7),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussNoise(p=0.3)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Paths
image_dir = "/content/dataset/train/images"
label_dir = "/content/dataset/train/labels"
output_img_dir = "/content/dataset/augmented_images"
output_lbl_dir = "/content/dataset/augmented_labels"

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_lbl_dir, exist_ok=True)

# Process each image
for img_path in glob(os.path.join(image_dir, "*.jpg")):
    img = cv2.imread(img_path)
    h, w, _ = img.shape
    filename = os.path.basename(img_path).replace('.jpg', '')

    label_path = os.path.join(label_dir, f"{filename}.txt")
    if not os.path.exists(label_path): continue

    with open(label_path, "r") as f:
        lines = f.read().splitlines()

    boxes = []
    class_labels = []
    for line in lines:
        cls, x, y, bw, bh = map(float, line.strip().split())
        box = yolo_to_albumentations([x, y, bw, bh], w, h)
        boxes.append(box)
        class_labels.append(int(cls))

    for i in range(10):  # 5 augmentations per image
        augmented = transform(image=img, bboxes=boxes, class_labels=class_labels)
        aug_img = augmented["image"]
        aug_boxes = augmented["bboxes"]
        aug_labels = augmented["class_labels"]

        out_img_name = f"{filename}_aug_{i}.jpg"
        out_lbl_name = f"{filename}_aug_{i}.txt"

        cv2.imwrite(os.path.join(output_img_dir, out_img_name), aug_img)

        with open(os.path.join(output_lbl_dir, out_lbl_name), "w") as f:
            for box, cls in zip(aug_boxes, aug_labels):
                yolo_box = albumentations_to_yolo(box, w, h)
                f.write(f"{cls} {' '.join(map(str, yolo_box))}\n")


In [ ]:
import os
import shutil
from glob import glob

# Original folders
orig_img_dir = "/content/dataset/train/images"
orig_lbl_dir = "/content/dataset/train/labels"

# Augmented folders
aug_img_dir = "/content/dataset/augmented_images"
aug_lbl_dir = "/content/dataset/augmented_labels"

# Combined output folders
combined_img_dir = "/content/dataset/combined/train/images"
combined_lbl_dir = "/content/dataset/combined/train/labels"

# Make sure combined folders exist
os.makedirs(combined_img_dir, exist_ok=True)
os.makedirs(combined_lbl_dir, exist_ok=True)

# Copy original images
for file in glob(os.path.join(orig_img_dir, "*.jpg")):
    shutil.copy(file, combined_img_dir)

# Copy original labels
for file in glob(os.path.join(orig_lbl_dir, "*.txt")):
    shutil.copy(file, combined_lbl_dir)

# Copy augmented images
for file in glob(os.path.join(aug_img_dir, "*.jpg")):
    shutil.copy(file, combined_img_dir)

# Copy augmented labels
for file in glob(os.path.join(aug_lbl_dir, "*.txt")):
    shutil.copy(file, combined_lbl_dir)

print("✅ All original + augmented data combined successfully!")

✅ All original + augmented data combined successfully!


In [ ]:
import shutil

# Zip the combined dataset folder
shutil.make_archive("/content/combined_dataset", 'zip', "/content/dataset/combined")

print("✅ Zipped as /content/combined_dataset.zip")

✅ Zipped as /content/combined_dataset.zip
